<a href="https://colab.research.google.com/github/Z0rr09/OD-WeaponDetection/blob/master/Notebook_module_3%20v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries
#!pip install torch torchvision

# Import libraries
import torch
from torchvision import transforms, datasets
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torch.utils.data import DataLoader
import torchvision.transforms.functional as F
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define dataset paths
image_folder = "https://github.com/ari-dasci/OD-WeaponDetection/raw/main/Pistol%20detection/Weapons"
annotation_folder = "https://github.com/ari-dasci/OD-WeaponDetection/raw/main/Pistol%20detection/xmls"

# Define transform
transform = transforms.Compose([
    transforms.ToTensor()
])

# Load dataset
dataset = datasets.VOCDetection(root=image_folder, year='2007', image_set='train', download=True, transform=transform)

In [ ]:
#Get a few Example of database
def show_bbox(image_path):
  # convert image path to label path
  label_path = image_path.replace('/images/', '/labels/')
  label_path = label_path.replace('.jpg', '.txt')
  # Open the image and create ImageDraw object for drawing
  image = Image.open(image_path)
  draw = ImageDraw.Draw(image)
  print(image)
  with open(label_path, 'r') as f:
    for line in f.readlines():
      # Split the line into five values
      label, x, y, w, h = line.split(' ')

      # Convert string into float
      x = float(x)
      y = float(y)
      w = float(w)
      h = float(h)

      # Convert center position, width, height into
      # top-left and bottom-right coordinates
      W, H = image.size
      x1 = (x - w/2) * W
      y1 = (y - h/2) * H
      x2 = (x + w/2) * W
      y2 = (y + h/2) * H

      # Draw the bounding box with red lines
      draw.rectangle((x1, y1, x2, y2),outline=(255, 0, 0), width=5)# Line width
  return image


In [ ]:
import os
import xml.etree.ElementTree as ET
from PIL import Image
from torchvision import transforms

def parse_xml_annotation(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Extract image path
    image_path = root.find('path').text

    # Extract object bounding box coordinates and class labels
    boxes = []
    labels = []
    for obj in root.findall('object'):
        xmin = int(obj.find('bndbox').find('xmin').text)
        ymin = int(obj.find('bndbox').find('ymin').text)
        xmax = int(obj.find('bndbox').find('xmax').text)
        ymax = int(obj.find('bndbox').find('ymax').text)
        label = obj.find('name').text
        boxes.append([xmin, ymin, xmax, ymax])
        labels.append(label)

    return image_path, boxes, labels

def my_collate(batch):
    images = []
    targets = []

    for xml_file in batch:
        # Parse XML annotation file
        image_path, boxes, labels = parse_xml_annotation(xml_file)

        # Load image
        image = Image.open(image_path).convert("RGB")

        # Apply transformations
        transform = transforms.Compose([
            transforms.Resize((500, 500)),
            transforms.ToTensor(),
        ])
        image = transform(image)

        images.append(image)
        targets.append({'boxes': boxes, 'labels': labels})

    return torch.stack(images), targets




In [ ]:
# Example usage:
annotation_folder = "/content/OD-WeaponDetection/OD-WeaponDetection-master/Pistol detection/xmls"
xml_files = [os.path.join(annotation_folder, file) for file in os.listdir(annotation_folder)]

data_loader = DataLoader(xml_files, batch_size=16, shuffle=True, collate_fn=my_collate)



# Load pre-trained Faster R-CNN model
model = fasterrcnn_resnet50_fpn( weights='DEFAULT')
model.to(device)

# Define optimizer and loss function
optimizer = torch.optim.SGD(model.parameters(), lr=0.03, momentum=0.9)
criterion = torch.nn.SmoothL1Loss()

In [ ]:
################################# Get a few examples of Photos to present ####################################
base_Directory = "/content/drive/MyDrive/DeepLData/obj_train_data/images/train/"
all_files = os.listdir(base_Directory)
image_Paths = []

#Get 9 images randomly to see some images (train)
for i in range(9):
  img = random.choice(all_files)
  img_Path = base_Directory + img
  image_Paths.append(img_Path)

#Create Subplots axes to present data
fig, axes = plt.subplots(3, 3, figsize=(10, 10))
axes = axes.flatten()

#Images to train the data
for i, (ax, image_path) in enumerate(zip(axes, image_Paths)):
  img = show_bbox(image_path)
  ax.imshow(img)

In [ ]:

# Load dataset with a subset
subset_size = 500  # Change this to the desired subset size
subset_dataset = datasets.VOCDetection(root=image_folder, year='2007', image_set='train', download=True, transform=transform)
subset_dataset = torch.utils.data.Subset(subset_dataset, range(subset_size))

# Define data loader with custom collate function
subset_data_loader = DataLoader(subset_dataset, batch_size=16, shuffle=True, num_workers=2, collate_fn=my_collate)

In [ ]:
# ... (previous code)

# Training loop (light training for demonstration purposes)
num_epochs = 50
for epoch in range(num_epochs):
    for images, targets in data_loader:
        images = list(image.to(device) for image in images)

        # Modify targets to include necessary keys (assuming bounding box annotations and labels)
        targets = [{k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in t.items()} for t in targets]
        for target in targets:
            if "boxes" not in target:
                target["boxes"] = torch.zeros((0, 4), dtype=torch.float32, device=device)
            if "labels" not in target:
                target["labels"] = torch.zeros((0,), dtype=torch.int64, device=device)

        optimizer.zero_grad()
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        losses.backward()
        optimizer.step()

# ... (continue with the rest of the script)


In [ ]:
# Visualize a sample prediction
model.eval()
with torch.no_grad():
    sample_image, sample_target = dataset[0]
    sample_image = sample_image.unsqueeze(0).to(device)
    prediction = model([sample_image])[0]

# Display the original image and the predicted bounding boxes
image = F.to_pil_image(sample_image.cpu().squeeze())
draw = ImageDraw.Draw(image)
draw.rectangle(prediction['boxes'][0].cpu().numpy(), outline="red")

plt.imshow(image)
plt.show()


In [6]:
import os
import requests
import zipfile
from io import BytesIO

# Define the GitHub repository URL
repo_url = 'https://github.com/ari-dasci/OD-WeaponDetection/archive/master.zip'

# Download the zip file from GitHub
response = requests.get(repo_url)
zip_file = zipfile.ZipFile(BytesIO(response.content))

# Specify the target directory in Colab
target_dir = '/content/OD-WeaponDetection'

# Extract the contents of the zip file to the target directory
zip_file.extractall(target_dir)

# Identify the actual directory containing the dataset
dataset_dir = os.path.join(target_dir, 'OD-WeaponDetection-master', 'Pistol detection')

# Print the contents of the dataset directory
print(os.listdir(dataset_dir))




['Weapons', 'xmls']


In [4]:


# Import necessary libraries
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split

# Set random seed for reproducibility
tf.random.set_seed(42)

# Define root directory
root_dir = '/content/OD-WeaponDetection/OD-WeaponDetection-master/Pistol detection'

# Define paths to your dataset
images_dir = os.path.join(root_dir, 'Weapons')
annotations_dir = '/content/OD-WeaponDetection/OD-WeaponDetection-master/Pistol detection/xmls'

# Image parameters
img_height, img_width = 150, 150
batch_size = 32

# Create a list of image files
image_files = [os.path.join(images_dir, file) for file in os.listdir(images_dir) if file.endswith('.jpg')]

# Create a list of annotation files
annotation_files = [os.path.join(annotations_dir, file) for file in os.listdir(annotations_dir) if file.endswith('.xml')]

# Ensure that corresponding image and annotation files exist
image_files = [file for file in image_files if os.path.splitext(file)[0] + '.xml' in annotation_files]

# Split the dataset into training and testing sets
train_image_files, test_image_files = train_test_split(image_files, test_size=0.2, random_state=42)

# Create data generators with augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_list(
    train_image_files,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_list(
    test_image_files,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)



ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
# Build the CNN model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // batch_size,
                    epochs=10,
                    validation_data=test_generator,
                    validation_steps=test_generator.samples // batch_size)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f'Test Accuracy: {test_acc * 100:.2f}%')

# Plot training history
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

ca va ?


100%|██████████| 460032000/460032000 [00:13<00:00, 35030423.01it/s]


Extracting https://github.com/ari-dasci/OD-WeaponDetection/raw/main/Pistol%20detection/Weapons/VOCtrainval_06-Nov-2007.tar to https://github.com/ari-dasci/OD-WeaponDetection/raw/main/Pistol%20detection/Weapons


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/DeepLData/obj_train_data/images/train/'

In [15]:

# Define custom collate function
def my_collate(batch):
    images = [item[0] for item in batch]
    targets = [item[1] for item in batch]

    # Resize images to a common size (e.g., 500x500)
    images = [F.resize(img, (500, 500)) for img in images]

    return torch.stack(images), targets

# Define data loader with custom collate function
data_loader = DataLoader(dataset, batch_size=2, shuffle=True, num_workers=2, collate_fn=my_collate)



# Load pre-trained Faster R-CNN model
model = fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT)
model.to(device)

# Define optimizer and loss function
optimizer = torch.optim.SGD(model.parameters(), lr=0.03, momentum=0.9)
criterion = torch.nn.SmoothL1Loss()

NameError: name 'FasterRCNN_ResNet50_FPN_Weights' is not defined

In [13]:
subset_data_loader


Using downloaded and verified file: https://github.com/ari-dasci/OD-WeaponDetection/raw/main/Pistol%20detection/Weapons/VOCtrainval_06-Nov-2007.tar
Extracting https://github.com/ari-dasci/OD-WeaponDetection/raw/main/Pistol%20detection/Weapons/VOCtrainval_06-Nov-2007.tar to https://github.com/ari-dasci/OD-WeaponDetection/raw/main/Pistol%20detection/Weapons


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Rob\\Desktop\\Definitiva\\armas (2575).jpg'

In [24]:
# Training loop (light training for demonstration purposes)
num_epochs = 2
for epoch in range(num_epochs):
    for images, targets in subset_data_loader:
        images = list(image.to(device) for image in images)

        # Modify targets to include necessary keys (assuming bounding box annotations and labels)
        targets = [{k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in t.items()} for t in targets]
        for target in targets:
            if "boxes" not in target:
                target["boxes"] = torch.zeros((0, 4), dtype=torch.float32, device=device)
            if "labels" not in target:
                target["labels"] = torch.zeros((0,), dtype=torch.int64, device=device)

        optimizer.zero_grad()
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        losses.backward()
        optimizer.step()

# ... (continue with the rest of the script

TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 54, in fetch
    return self.collate_fn(data)
  File "<ipython-input-21-8b1c1f8db139>", line 33, in my_collate
    image_path, boxes, labels = parse_xml_annotation(xml_file)
  File "<ipython-input-21-8b1c1f8db139>", line 7, in parse_xml_annotation
    tree = ET.parse(xml_file)
  File "/usr/lib/python3.10/xml/etree/ElementTree.py", line 1222, in parse
    tree.parse(source, parser)
  File "/usr/lib/python3.10/xml/etree/ElementTree.py", line 569, in parse
    source = open(source, "rb")
TypeError: expected str, bytes or os.PathLike object, not tuple


ValueError: images is expected to be a list of 3d tensors of shape [C, H, W], got torch.Size([1, 3, 333, 500])

In [26]:
# ... (previous code)

# Evaluation loop
model.eval()
all_predictions = []
all_targets = []

with torch.no_grad():
    for images, targets in data_loader:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        prediction = model(images)
        all_predictions.extend(prediction)
        all_targets.extend(targets)

# Calculate Average Precision
from sklearn.metrics import average_precision_score

y_true = []
y_scores = []

for target, prediction in zip(all_targets, all_predictions):
    # Convert box coordinates to numpy arrays for easier handling
    target_boxes = target['boxes'].cpu().numpy()
    prediction_boxes = prediction['boxes'].cpu().numpy()

    # Calculate IoU (Intersection over Union) between target and prediction boxes
    iou = calculate_iou(target_boxes, prediction_boxes)

    # Determine true positives (IoU > threshold)
    true_positives = (iou > 0.5).sum()

    # Append ground truth and prediction information
    y_true.append(torch.ones(true_positives))
    y_true.append(torch.zeros(len(target_boxes) - true_positives))

    y_scores.append(prediction['scores'])
    y_scores.append(torch.zeros(len(target_boxes) - true_positives))

y_true = torch.cat(y_true)
y_scores = torch.cat(y_scores)

# Calculate Average Precision using sklearn
average_precision = average_precision_score(y_true.numpy(), y_scores.numpy())
print(f'Average Precision: {average_precision}')

# Visualize predictions on a few test images
model.eval()
with torch.no_grad():
    for i in range(3):  # Visualize predictions for the first 3 images
        sample_image, sample_target = dataset[i]
        sample_image = sample_image.unsqueeze(0).to(device)
        prediction = model([sample_image])[0]

        # Display the original image and the predicted bounding boxes
        image = F.to_pil_image(sample_image.cpu().squeeze())
        draw = ImageDraw.Draw(image)

        for box, score in zip(prediction['boxes'], prediction['scores']):
            if score > 0.5:  # Confidence threshold
                draw.rectangle(box.cpu().numpy(), outline="red")

        plt.imshow(image)
        plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Rob\\Desktop\\Definitiva\\armas (1006).jpg'